In [3]:
#import libs
import sweetviz as sv
import pandas as pd
from pathlib import Path  
import sqlite3

In [4]:
data=pd.read_excel("CompassusData.xlsx")

In [5]:
data['ceas_ascode'] = pd.to_numeric(data['ceas_ascode'], errors='coerce')

In [6]:
sel_cols = list(data.select_dtypes(include='object'))
data[sel_cols] = data[sel_cols].fillna('').astype(str)

In [7]:
advert_report = sv.analyze(data,pairwise_analysis='off')

In [8]:
advert_report.show_html('CompassusData.html')

Report CompassusData.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [9]:
for col in data:
    if (data[col].isna().sum() / 22780)*100 == 100.0:
        data.drop(col, inplace=True, axis = 1)

data.columns.size

151

In [10]:
filepath = Path('data_filtered.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
data.to_csv(filepath)  

In [11]:
def getColumnDtypes(dataTypes):
    dataList = []
    for x in dataTypes:
        if(x == 'int64'):
            dataList.append('int')
        elif (x == 'float64'):
            dataList.append('float')
        elif (x == 'bool'):
            dataList.append('boolean')
        else:
            dataList.append('varchar')
    return dataList

In [12]:
columnName = list(data.columns.values)
columnDataType = getColumnDtypes(data.dtypes)

In [13]:
createTableStatement = 'CREATE TABLE IF NOT EXISTS data ('
for i in range(len(columnDataType)):
    createTableStatement = createTableStatement + '\n' + columnName[i] + ' ' + columnDataType[i] + ','
createTableStatement = createTableStatement[:-1] + ' );'

In [16]:
database=Path('compassus_data.db')
if database.is_file():
    database.unlink()
database.touch()
conn = sqlite3.connect('compassus_data.db')
c = conn.cursor()
c.execute(createTableStatement)

data.to_sql('data', conn, if_exists='append', index = False)